# Environment Initialization

In [1]:
import matplotlib.pyplot as plt
%matplotlib widget

import pandas as pd
import numpy as np

from statsmodels.tsa.arima.model import ARIMA

import tensorflow as tf
from tensorflow import keras

from typing import Optional

import os
import shutil
if not os.path.exists("./report/figures/"):
  os.makedirs("./report/figures/")
if not os.path.exists("./models/"):
  os.makedirs("./models/")

2023-11-23 08:47:06.619848: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-23 08:47:06.709748: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 08:47:06.709817: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 08:47:06.709838: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-23 08:47:06.773960: I tensorflow/core/platform/cpu_feature_g

# Data Import/Pre-Processing/Visualization

In [2]:
dtype = {
  "Datetime": "string",
  "solar_mw": np.float32,
  "wind-direction": np.float32,
  "wind-speed": np.float32,
  "humidity": np.float32,
  "average-wind-speed-(period)": np.float32,
  "average-pressure-(period)": np.float32,
  "temperature": np.float32
}
solar_energy_df = pd.read_csv("./solarenergy.csv",
                              delimiter=",",
                              dtype=dtype)

solar_energy_df["Datetime"] = pd.to_datetime(solar_energy_df["Datetime"],
                                             format="%d/%m/%Y %H:%M")
solar_energy_df = solar_energy_df.set_index("Datetime").sort_index()

axes = solar_energy_df.plot(figsize=(8,5), subplots=True, sharex=True)
plt.suptitle("Solar Energy Dataset (Raw)")
axes[-1].set_xlabel("Time")
for axis in axes:
  axis.autoscale(True, "x", True)
plt.savefig("./report/figures/timeseries_raw.svg")
plt.close("all")

solar_energy_df = solar_energy_df.dropna()
solar_energy_df = solar_energy_df.resample("1H").interpolate("linear")

axes = solar_energy_df.plot(figsize=(8,5), subplots=True, sharex=True)
plt.suptitle("Solar Energy Dataset (Clean)")
axes[-1].set_xlabel("Time")
for axis in axes:
  axis.autoscale(True, "x", True)
plt.savefig("./report/figures/timeseries_clean.svg")
plt.close("all")

solar_energy_df = \
  (solar_energy_df - solar_energy_df.mean()) / solar_energy_df.std()

axes = solar_energy_df.plot(figsize=(8,5), subplots=True, sharex=True)
plt.suptitle("Solar Energy Dataset (Normalized)")
axes[-1].set_xlabel("Time")
for axis in axes:
  axis.autoscale(True, "x", True)
plt.savefig("./report/figures/timeseries_norm.svg")
plt.close("all")

# Baseline Model

In [3]:
training_ratio = 0.7
training_limit = \
  solar_energy_df.index[
      int(training_ratio*solar_energy_df.shape[0])
    ]

best_rmse = np.inf
arimax_model = None

x_train_df = solar_energy_df.iloc[
                :int(training_ratio*solar_energy_df.shape[0]),
                solar_energy_df.columns != "solar_mw"
              ]
y_train_df = solar_energy_df.loc[:training_limit-pd.Timedelta(hours=1),
                                  "solar_mw"]

x_test_df = solar_energy_df.iloc[
              int(training_ratio*solar_energy_df.shape[0]):,1:
            ]
y_test_df = solar_energy_df.loc[training_limit:, "solar_mw"]

for p in [3]:
  for d in [0]:
    for q in [4]:
      try:
        new_model = ARIMA(
          endog=y_train_df,
          order=(p,d,q)
        ).fit()
      except:
        continue

      new_prediction = new_model.get_forecast(
          steps=solar_energy_df.index[-1]
        ).predicted_mean.to_numpy()
      
      new_rmse = np.sqrt(np.mean((new_prediction-y_test_df.to_numpy())**2))
      if new_rmse < best_rmse:
        best_rmse = new_rmse
        arimax_model = new_model

baseline_fit = arimax_model.fittedvalues

baseline_prediction = arimax_model.get_forecast(
    steps=solar_energy_df.index[-1]
  ).predicted_mean

print(arimax_model.summary())
p,d,q = arimax_model.specification["order"]
print(f"Baseline RMSE: {best_rmse}")
print(f"Baseline MAE: {np.abs(baseline_prediction-y_test_df.to_numpy()).max()}")

plt.figure(figsize=(8,5))
plt.plot(solar_energy_df["solar_mw"],
         label="Measured")
plt.plot(baseline_fit, label="Fitted")
plt.plot(baseline_prediction, label="Estimated")
plt.autoscale(True, "x", tight=True)
plt.title(f"ARIMAX({p},{d},{q})")
plt.legend()
plt.savefig("./report/figures/baseline_fit.svg")
plt.close("all")

                               SARIMAX Results                                
Dep. Variable:               solar_mw   No. Observations:                 1544
Model:                 ARIMA(3, 0, 4)   Log Likelihood                1960.798
Date:                Thu, 23 Nov 2023   AIC                          -3903.597
Time:                        08:47:11   BIC                          -3855.518
Sample:                    05-03-2020   HQIC                         -3885.712
                         - 07-06-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0839      0.330     -0.254      0.799      -0.731       0.563
ar.L1          2.9201      0.005    574.051      0.000       2.910       2.930
ar.L2         -2.9087      0.010   -294.849      0.0

/home/sergiovaneg/miniforge3/envs/ADAML/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


# Recursive Models

## Data Preparation

In [4]:
n_feats = 1
p = 10
q = 48

def tensor_memory_reshaper(in_np:np.ndarray, out_np:Optional[np.ndarray],
                           memory:int, n_feats_internal:int = n_feats):
  in_np = in_np.reshape((-1,1,n_feats_internal))
  for _ in range(memory):
    next_np = in_np[1:,-1,:].reshape(((-1,1,n_feats_internal)))
    in_np = np.concatenate((in_np[:-1,:,:], next_np), axis=1)

  if out_np is not None:
    return (tf.convert_to_tensor(in_np),
            tf.convert_to_tensor(out_np[memory:]))
  else:
    return (tf.convert_to_tensor(in_np), None)

x_train_np = np.pad(
    y_train_df[:-1].to_numpy().reshape((-1,1)), ((q+1,0),(0,0)), mode="edge"
  )
y_train_np = np.pad(y_train_df.to_numpy(), ((q,0),), mode="edge")

x_test_np = pd.concat(
    (y_train_df.iloc[-(q+1):], y_test_df[:-1])
  ).to_numpy().reshape((-1,1))
y_test_np = pd.concat((y_train_df.iloc[-q:], y_test_df)).to_numpy()

x_train_tensor,y_train_tensor = \
  tensor_memory_reshaper(x_train_np, y_train_np, q)
_,y_test_tensor = \
  tensor_memory_reshaper(x_test_np, y_test_np, q)

class StateResetCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs=None):
    self.model.reset_states()

2023-11-23 08:47:12.956310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 08:47:13.027436: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 08:47:13.027815: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Simple RNN

In [5]:
if not os.path.exists("./models/rnn_model.keras"):
  rnn_model = keras.Sequential()
  rnn_model.add(keras.layers.SimpleRNN(p, stateful=True,
                                       batch_input_shape=(1,q+1,n_feats)))
  rnn_model.add(keras.layers.Dense(1, keras.activations.linear))

  rnn_model_checkpoint = keras.callbacks.ModelCheckpoint(
                            "./models/rnn_model_checkpoint",
                            monitor="loss",
                            save_best_only=True,
                            save_weights_only=False
                          )
  rnn_model_early_stop = keras.callbacks.EarlyStopping(
                            monitor="val_loss",
                            patience=10,
                            restore_best_weights=True
                          )

  rnn_model.compile(optimizer="Adam", loss="mse")
  rnn_model.fit(x_train_tensor, y_train_tensor,
                shuffle=False, batch_size=1,
                epochs=100, verbose="auto",
                validation_split=0.2,
                callbacks=[rnn_model_early_stop, StateResetCallback()])
  
  if os.path.exists("./models/rnn_model_checkpoint"):
    rnn_model = keras.models.load_model("./models/rnn_model_checkpoint")
    shutil.rmtree("./models/rnn_model_checkpoint")
  rnn_model.save("./models/rnn_model.keras")
else:
  rnn_model = keras.models.load_model("./models/rnn_model.keras")

rnn_model.reset_states()
rnn_fit = rnn_model.predict(x_train_tensor, batch_size=1).flatten()

rnn_prediction = np.zeros(y_test_tensor.shape)
buffer = y_train_np[-(q+1):]
for idx in range(len(y_test_tensor)):
  current_input,_ = tensor_memory_reshaper(buffer, None, q, 1)
  rnn_prediction[idx] = rnn_model.predict(current_input, batch_size=1)[-1]

  buffer[:-1] = buffer[1:]
  buffer[-1] = rnn_prediction[idx]

print(f"RNN RMSE: {np.sqrt(np.mean((rnn_prediction-y_test_tensor)**2))}")
print(f"RNN Max AE: {np.abs(rnn_prediction-y_test_tensor).max()}")

plt.figure(figsize=(8,5))
plt.plot(solar_energy_df["solar_mw"],
         label="Measured")
plt.plot(y_train_df.index, rnn_fit, label="Fitted")
plt.plot(y_test_df.index, rnn_prediction, label="Estimated")
plt.autoscale(True, "x", tight=True)
plt.title(f"Simple RNN")
plt.legend()
plt.savefig("./report/figures/rnn_fit.svg")
plt.close("all")

2023-11-23 08:47:13.475287: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 0s 12ms/step


/tmp/ipykernel_5522/1728977838.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rnn_prediction[idx] = rnn_model.predict(current_input, batch_size=1)[-1]


1/1 [==============================] - 0s 11ms/step
RNN RMSE: 1.300902247428894
RNN Max AE: 3.654114246368408


## LSTM

In [6]:
if not os.path.exists("./models/lstm_model.keras"):
  lstm_model = keras.Sequential()
  lstm_model.add(keras.layers.LSTM(p, stateful=True,
                                   batch_input_shape=(1,q+1,n_feats)))
  lstm_model.add(keras.layers.Dense(1, keras.activations.linear))

  lstm_model_checkpoint = keras.callbacks.ModelCheckpoint(
                            "./models/lstm_model_checkpoint",
                            monitor="loss",
                            save_best_only=True,
                            save_weights_only=False
                          )
  lstm_model_early_stop = keras.callbacks.EarlyStopping(
                            monitor="val_loss",
                            patience=10,
                            restore_best_weights=True
                          )

  lstm_model.compile(optimizer="Adam", loss="mse")
  lstm_model.fit(x_train_tensor, y_train_tensor,
                shuffle=False, batch_size=1,
                epochs=100, verbose="auto",
                validation_split=0.2,
                callbacks=[lstm_model_early_stop, StateResetCallback()])
  
  if os.path.exists("./models/lstm_model_checkpoint"):
    lstm_model = keras.models.load_model("./models/lstm_model_checkpoint")
    shutil.rmtree("./models/lstm_model_checkpoint")
  lstm_model.save("./models/lstm_model.keras")
else:
  lstm_model= keras.models.load_model("./models/lstm_model.keras")

lstm_model.reset_states()
lstm_fit = lstm_model.predict(x_train_tensor, batch_size=1).flatten()

lstm_prediction = np.zeros(y_test_tensor.shape)
buffer = y_train_np[-(q+1):]
for idx in range(len(y_test_tensor)):
  current_input,_ = tensor_memory_reshaper(buffer, None, q, 1)
  lstm_prediction[idx] = lstm_model.predict(current_input, batch_size=1)

  buffer[:-1] = buffer[1:]
  buffer[-1] = lstm_prediction[idx]

print(f"LSTM RMSE: {np.sqrt(np.mean((lstm_prediction-y_test_tensor)**2))}")
print(f"LSTM Max AE: {np.abs(lstm_prediction-y_test_tensor).max()}")

plt.figure(figsize=(8,5))
plt.plot(solar_energy_df["solar_mw"],
         label="Measured")
plt.plot(y_train_df.index, lstm_fit, label="Fitted")
plt.plot(y_test_df.index, lstm_prediction, label="Estimated")
plt.autoscale(True, "x", tight=True)
plt.title(f"LSTM")
plt.legend()
plt.savefig("./report/figures/lstm_fit.svg")
plt.close("all")

  47/1544 [..............................] - ETA: 5s

2023-11-23 08:47:46.772895: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8800


1/1 [==============================] - 0s 11ms/step


/tmp/ipykernel_5522/69328896.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  lstm_prediction[idx] = lstm_model.predict(current_input, batch_size=1)


1/1 [==============================] - 0s 14ms/step
LSTM RMSE: 1.3487011194229126
LSTM Max AE: 3.07865834236145
